In [1]:
print('========================================================================')
print('Python WRF-GHG: IC from CAMS')
print('Michal Galkowski, MPI-BGC Jena')
print('modified by David Ho, MPI-BGC Jena')
print('For handling CAMS product from Santiago,')
print('Translated for python by Noelia Rojas, IFUSP Brazil')
print('over the Amazon domain.')


Python WRF-GHG: IC from CAMS
Michal Galkowski, MPI-BGC Jena
modified by David Ho, MPI-BGC Jena
For handling CAMS product from Santiago,
Translated for python by Noelia Rojas, IFUSP Brazil
over the Amazon domain.


In [3]:
import numpy as np
import pandas as pd

In [5]:
wrfinput_dir_path    ='/media/nonna/TOSHIBA EXT/2023/IFUSP/DATA/WRF/wrfinput/'
CAMS_data_dir_path   ='/media/nonna/TOSHIBA EXT/2023/IFUSP/DATA/sources/Boundary/CAMS/'
IFS_L137_ab_file     = '/media/nonna/TOSHIBA EXT/2023/IFUSP/DATA/sources/Boundary/ecmwf_coeffs_L137.csv'

sim_time        = '2022-08-01 00:00:00','2022-08-31 23:00:00' 
dates           = pd.to_datetime(sim_time[0]).strftime('%Y-%m-%d')
year            = pd.to_datetime(sim_time[0]).strftime('%Y')
month           = pd.to_datetime(sim_time[0]).strftime('%m')
day             = pd.to_datetime(sim_time[0]).strftime('%d')

In [6]:
#Setting important paths to files and directories
#CAMS_interpolation_indices_file_path = '/work/mj0143/b301043/4Flavio/matlab/cal_indices/results/Flavio_interp_indices.mat'
#%run -i interp_indices.py
requested_domains = [ "d01", "d02", "d03", "d04" ]

In [ ]:
#Calculate based on number of wrfinput files available:
wrf_i          = cdf.Dataset(wrf_inp)
wrfinput_path = fullfile( wrfinput_dir_path, 'wrfinput_d01' );


In [ ]:
% Set start time manually
%simstart_time = datetime('2018-06-06 18:00:00');
% OR
% Set start time automatically (read from wrfinput)
wrfinput_path = fullfile( wrfinput_dir_path, 'wrfinput_d01' );
%%wrfinput_path = sprintf( '%swrfinput_d01_%4d-%02d-%02d', realoutput_path, year ,month, day );
simstart_time = ncread( wrfinput_path, 'Times' );
simstart_time = datetime( ncread( wrfinput_path, 'Times' )', 'InputFormat', 'yyyy-MM-dd_HH:mm:ss' );
simstart_time.TimeZone = 'UTC';
simstart_time.Format = 'yyyMMdd';

In [ ]:







%display('Loading in the pre-calculated nearest-neighbour interipolation indices.');
%load( CAMS_interpolation_indices_file_path );
% Should have loaded objects like: cams_indices_d01, cams_indices_d02
% Every object describes indices for each domain.

% Based on the initial day, set the CAMS filenames to be read.
path_CAMS_ml_file   = fullfile( strcat( CAMS_data_dir_path, 'CAMS_co_ch4_', char( simstart_time ), '.nc' ) );
path_CAMS_lnsp_file = fullfile( strcat( CAMS_data_dir_path, 'CAMS_co_ch4_', char( simstart_time ), '.nc' ) );

% Reading a and b parameters from the model config L137
fid=fopen( IFS_L137_ab_file, 'r' );
rawin=textscan(fid,'%d,%f,%f,%f,%f,%f,%f,%f,%f','Headerlines',1);
a=rawin{2};
b=rawin{3};
clear rawin;



% UI:
display(  sprintf('Getting CAMS latitudes and longitudes from:\n  %s', path_CAMS_ml_file )  );

cams_lat = ncread( path_CAMS_ml_file, 'latitude' );
%cams_lon = ncread( path_CAMS_ml_file, 'longitude' );
cams_lon = ncread( path_CAMS_ml_file, 'longitude' ) - 360;
[mesh_lat,mesh_lon] = meshgrid( cams_lat, cams_lon );
% Times are given as hours since 1900-01-01 00:00:00 UTC
cams_times = ncread( path_CAMS_ml_file, 'time' );
cams_dates = datetime('1900-01-01 00:00:00') + hours(cams_times);
%%cams_dates = datetime( sprintf( '%4d-%02d-%02d 00:00:00', year, month, day) ) + hours(cams_times);
cams_dates.TimeZone = 'UTC';

% Check if one of the times is the same as simstart time:
cams_time_idx = find( posixtime( cams_dates ) == posixtime( simstart_time ) );
simstart_time.Format = 'yyyy-MM-dd HH:mm:ss';
display( sprintf( 'CAMS file contains the values for %s at index no. %d', simstart_time, cams_time_idx ) );

% Important info on differences between sp and lnsp in IFS
% http://www.iup.uni-bremen.de/~hilboll/blog/2018/12/understanding-surface-pressure-in-ecmwf-era5-reanalysis-data/
display(  sprintf( 'Getting CAMS lnsp (ln of surface pressure) from:\n  %s', path_CAMS_lnsp_file )  );
nc_lnsp_start_vector = [ 1 1 1 cams_time_idx ];
nc_lnsp_count_vector = [ Inf Inf Inf 1 ];
cams_lnsp = ncread( path_CAMS_lnsp_file, 'lnsp', nc_lnsp_start_vector, nc_lnsp_count_vector );

% Warning! scaling factor and offset applied automatically when reading with ncread!
% https://ch.mathworks.com/help/matlab/ref/ncread.html?s_tid=srchtitle
% See output section
cams_pressure = exp( cams_lnsp );
% From ncdump -h:
% short lnsp(time, latitude, longitude) ;
% lnsp:scale_factor = 7.03116701535987e-06 ;
% lnsp:add_offset = 11.3388417374072 ;
% unpacked_value = packed_value * scale_factor + add_offset
% Conclusion: unpacking done by Matlab automatically!



% Before the loop, read in full CAMS fields.
nc_start_vector = [1 1 1 cams_time_idx];
nc_count_vector = [Inf Inf Inf 1];

cams_ch4 = ncread( path_CAMS_ml_file, 'ch4_c', nc_start_vector, nc_count_vector  ) * (28.97/16.01)*1e6;
cams_co  = ncread( path_CAMS_ml_file, 'co', nc_start_vector, nc_count_vector  ) * (28.97/28.01)*1e6;
%%cams_co2 = ncread( path_CAMS_ml_file, 'co2', nc_start_vector, nc_count_vector  ) * (28.97/44.01)*1e6;



% Now execute the caluclation per-domain
for domain_idx = 1:size(requested_domains,2)
%  domain_idx = 1;
  % UI:
  display( sprintf( 'Processing domain: %s', requested_domains( domain_idx ) ) );

  % Load CAMS interpolation indices for this domain
  display('Loading in the pre-calculated nearest-neighbour interipolation indices.');
  S = struct2cell( load( CAMS_interpolation_indices_file_path, strcat( 'cams_indices_', char( requested_domains(domain_idx ) ) ) ) );
  interpolation_indices = S{1};

  % Load wrfinput file for the current domain
  wrfinput_path = fullfile( wrfinput_dir_path, char( strcat( 'wrfinput_', requested_domains( domain_idx ) ) ) );

  wrf_xlat  = ncread( wrfinput_path, 'XLAT'  );
  wrf_xlong = ncread( wrfinput_path, 'XLONG' );

  % Set offsets as initial values of biogenic tracers
  dummy_3d_scalar_field = ncread( wrfinput_path, 'CO2_BIO' );
  n_vertical_levels = size( dummy_3d_scalar_field, 3 );
  n_ew              = size( dummy_3d_scalar_field, 1 );
  n_sn              = size( dummy_3d_scalar_field, 2 );

  co2_bio_init = 400*ones( size( dummy_3d_scalar_field ) );
  ch4_bio_init = 1.8*ones( size( dummy_3d_scalar_field ) );
%%  ch4_soil_uptake_init = 1.8*ones( size( dummy_3d_scalar_field ) );

  % Write the values already:
  ncwrite( wrfinput_path, 'CO2_BIO', co2_bio_init );
  ncwrite( wrfinput_path, 'CH4_BIO', ch4_bio_init );
%%  ncwrite( wrfinput_path, 'CH4_BIO_Soils', ch4_soil_uptake_init );

  % Proceed to CAMS fields
  % For vertical assignment, nearest neighbour method will be used.
  display( 'Calculating pressures' );

  wrf_pressure = ncread( wrfinput_path, 'PB' ) + ncread( wrfinput_path, 'P' );

  wrf_init_CH4_BCK = -999*ones( size( dummy_3d_scalar_field ) );
  wrf_init_CO_BCK  = -999*ones( size( dummy_3d_scalar_field ) );
%%  wrf_init_CO2_BCK = -999*ones( size( dummy_3d_scalar_field ) );

    
    % For each cell:
  for lat_idx = 1:n_sn

    display( sprintf( 'Processing latitude band %d/%d', lat_idx, n_sn ) );

    for lon_idx = 1:n_ew

      % Get CAMS surface pressure
      surface_pressure = cams_pressure( interpolation_indices( lon_idx, lat_idx, 1), ...
                                        interpolation_indices( lon_idx, lat_idx, 2) );
      % Calculate CAMS vertical pressures for the available levels
      %cams_v_pressures = surface_pressure * b(49:137) + a(49:137);
      cams_v_pressures = surface_pressure * b(:) + a(:);

      % Get WRF levels
      wrf_v_pressures  = squeeze( wrf_pressure( lon_idx, lat_idx, : ) );

      for lvl_idx = 1:n_vertical_levels
          
        difference = abs( cams_v_pressures - wrf_v_pressures(lvl_idx) );

        % min() added as there was a case where two levels had the same difference
        cams_nearest_lvl_idx = min( find( difference == min(difference) ) );

        cams_indices = [ interpolation_indices( lon_idx, lat_idx, 1)...
                         interpolation_indices( lon_idx, lat_idx, 2)...
                         cams_nearest_lvl_idx ];

        wrf_init_CH4_BCK( lon_idx, lat_idx, lvl_idx ) = cams_ch4( interpolation_indices( lon_idx, lat_idx, 1), interpolation_indices( lon_idx, lat_idx, 2), cams_nearest_lvl_idx  );
        wrf_init_CO_BCK( lon_idx, lat_idx, lvl_idx )  = cams_co( interpolation_indices( lon_idx, lat_idx, 1), interpolation_indices( lon_idx, lat_idx, 2), cams_nearest_lvl_idx );
%%        wrf_init_CO2_BCK( lon_idx, lat_idx, lvl_idx ) = cams_co2( interpolation_indices( lon_idx, lat_idx, 1), interpolation_indices( lon_idx, lat_idx, 2), cams_nearest_lvl_idx );

      end
    end
  end

display('Writing values from CAMS for CH4_BCK, CO_BCK and CO2_BCK fields of wrfinput:')
%%ncwrite( wrfinput_path, 'CO2_BCK', wrf_init_CO2_BCK );
ncwrite( wrfinput_path, 'CH4_BCK', wrf_init_CH4_BCK );
ncwrite( wrfinput_path, 'CO_BCK',  wrf_init_CO_BCK  );

end





% Following variables are passed into the script when it's called from bash.
%display( runpath );
%display( cams_path );
%display( operational_dir );
% Example values:
%runpath='/work/mj0143/b301033/Models/WRFChem_CoMet_Reanalysis_v2.1/WRFV3/run';
%cams_path='/work/mj0143/b301033/Data/CoMet_input/CAMS_0.125x0.125/Rv1v2';
%operational_dir='/work/mj0143/b301033/Projects/CoMet/JFS.Reanalysis_v2';

% Mike: Used forecast.schedule.file for automatisation. ========================
% A nasty matlab way of reading a csv:
%schedule_file_name = fullfile(operational_dir,'schedule.of.CoMet.WRF.run');
%schedule_file_id = fopen(schedule_file_name);
%schedule_array = textscan(schedule_file_id,'%s %s %s');
% Schedule array is a nasty matlab object, of which values are read
% as: xxx{column}{row), including header. Months column = 2, days column = 3
%fclose(schedule_file_id);
% Forecast schedule opened =====================================================


display('Script completed.');